# MSMatch on EuroSAT RGB

This notebook contains a condensed version of training a U-Net / EfficientNet-B0 on the EuroSAT RGB dataset using the semi-supervised MSMatch approach. For details on the method please refer to the paper. To run this notebook, please set up a conda environment with the provided environment.yml. 

In [11]:
# Utility imports
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

# Main imports
import torch
import MSMatch as mm
from MSMatch.utils.create_dir_string import create_dir_str
from MSMatch.utils.load_cfg import load_cfg
from termcolor import colored
from  MSMatch.utils.print_cfg import print_cfg

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


The next variable contains the path to the configuration file `.toml`.  If you set variable `cfg_path` was to `None`, the default configuration is used. Ohterwise, the cfg file speicified by the path is loaded.

In [3]:
cfg_path=None

In [41]:
# We use a cfg DotMap (a dictionary with dot accessors) to store the configuration for the run
if cfg_path is not None:
    print("Using configuration file: "+colored(cfg_path,"red"))
    cfg=load_cfg(cfg_path)
else:
    print("Using standard configuration...")
    cfg = mm.get_default_cfg()

print_cfg(cfg)

Using standard configuration...
dataset                     : eurosat_rgb    |net                         : unet           |batch_size                  : 32             |
p_cutoff                    : 0.95           |lr                          : 0.01           |uratio                      : 7              |
weight_decay                : 0.00075        |ulb_loss_ratio              : 1.0            |seed                        : 42             |
num_labels                  : 100            |opt                         : SGD            |pretrained                  : 0              |
save_dir                    : ./saved_models |save_name_root              : test           |ema_m                       : 0.999          |
bn_momentum                 : 0.0010000000000000009|eval_batch_size             : 1024           |momentum                    : 0.9            |
T                           : 0.5            |amp                         : 0              |hard_label                  : 1     

Apply new configuration.

In [ ]:
# Set seeds for reproducibility and enable loggers
mm.set_seeds(cfg.seed)
logger_level = "INFO"
logger = mm.get_logger(cfg.save_name, cfg.save_path, logger_level)
tb_log = mm.TensorBoardLog(cfg.save_path, "")

## Semi-supervised Learning (SSL) Datasets 
In MSMatch we utilize both labeled and unlabeled data. To facilitiate this there is a class called `SSL_Dataset` in the MSMatch module which wraps around the `BasicDataset` class to take care of this.

This is necessary as in each training iteration we provide a supervised and unsupervised loss term with different (stronger) augmentations for the unsupervised part. (see paper)

We set up one dataset for training and one for testing.

**Note**: For below cell to execute make sure you have placed the EuroSAT RGB dataset in a folder `data/EuroSAT_RGB` from the project root. (You should have folders like `AnnualCrop` etc inside above folder which contain the images for each class)

You can download the dataset [from GitHub](https://madm.dfki.de/files/sentinel/EuroSAT.zip).

In [ ]:
# Construct Dataset
print("Loading "+colored("train", "red")+ " dataset...")
train_dset = mm.SSL_Dataset(
    name=cfg.dataset, train=True, data_dir=None, seed=cfg.seed,
)
lb_dset, ulb_dset = train_dset.get_ssl_dset(cfg.num_labels)

cfg.num_classes = train_dset.num_classes
cfg.num_channels = train_dset.num_channels

print("Loading "+colored("eval", "blue")+ " dataset...")
_eval_dset = mm.SSL_Dataset(
    name=cfg.dataset, train=False, data_dir=None, seed=cfg.seed,
)
eval_dset = _eval_dset.get_dset()

This project contains two different architectures for the backbone classifier, a very small U-Net-style encoder and the standard EfficientNet. Specify either `unet` or `efficientnet-b0` (up to `b7`) in `cfg.net` to get the specific model.

In [ ]:
print("Initializing ", cfg.net)
net_builder = mm.get_net_builder(
    cfg.net,
    pretrained=cfg.pretrained,
    in_channels=cfg.num_channels,
    scale=cfg.scale
)

The main model we call `FixMatch`. It differs from pure backbone in sofar that it also implements features like the exponential moving average of the backbone weights and takes care of computing the different loss terms. 

In [ ]:
model = mm.FixMatch(
        net_builder,
        cfg.num_classes,
        cfg.num_channels,
        cfg.ema_m,
        T=cfg.T,
        p_cutoff=cfg.p_cutoff,
        lambda_u=cfg.ulb_loss_ratio,
        hard_label=True,
        num_eval_iter=cfg.num_eval_iter,
        tb_log=tb_log,
        logger=logger,
    )
logger.info(f"Number of Trainable Params: {sum(p.numel() for p in model.train_model.parameters() if p.requires_grad)}")

Specify number of epochs to train, for convergence a value like 100 may be sensible. **Please, note:** the number of training epochs will not match `cfg.epoch` for `cfg.batch_size` different from 32. This is done to keep the number of images used during the whole training constant regardless of the batch size used.

In [ ]:
# Number of training iterations is based on that and how regularly we evaluate the model.
# Note that batch size here only refers to the supervised part, so the real batch size
# is cfg.batch_size * (1 + cfg.ulb_ratio)
cfg.num_train_iter = cfg.epoch * cfg.num_eval_iter * 32 // cfg.batch_size

In [ ]:
# get optimizer, ADAM and SGD are supported.
optimizer = mm.get_optimizer(
    model.train_model, cfg.opt, cfg.lr, cfg.momentum, cfg.weight_decay
)
# We use a learning rate schedule to control the learning rate during training.
scheduler = mm.get_cosine_schedule_with_warmup(
    optimizer, cfg.num_train_iter, num_warmup_steps=cfg.num_train_iter * 0
)
model.set_optimizer(optimizer, scheduler)

# If a CUDA capable GPU is used, we move everything to the GPU now
if torch.cuda.is_available():
    cfg.gpu = 0
    torch.cuda.set_device(cfg.gpu)
    model.train_model = model.train_model.cuda(cfg.gpu)
    model.eval_model = model.eval_model.cuda(cfg.gpu)

logger.info(f"model_arch: {model}")
logger.info(f"Arguments: {cfg}")

Originally the codebase supports parallel data augmentation and distributed learning on multiple GPUs in one machine. The below code is a bit of a relic of that. In practice it takes of creating a generator to get a batch of augmented training images for labeled (lb) and unlabled (ulb) data.

In [ ]:
loader_dict = {}
dset_dict = {"train_lb": lb_dset, "train_ulb": ulb_dset, "eval": eval_dset}

loader_dict["train_lb"] = mm.get_data_loader(
    dset_dict["train_lb"],
    cfg.batch_size,
    data_sampler="RandomSampler",
    num_iters=cfg.num_train_iter,
    num_workers=1,
    distributed=False,
)

loader_dict["train_ulb"] = mm.get_data_loader(
    dset_dict["train_ulb"],
    cfg.batch_size * cfg.uratio,
    data_sampler="RandomSampler",
    num_iters=cfg.num_train_iter,
    num_workers=4,
    distributed=False,
)

loader_dict["eval"] = mm.get_data_loader(
    dset_dict["eval"], cfg.eval_batch_size, num_workers=1
)

## set DataLoader on FixMatch
model.set_data_loader(loader_dict)

Now we are ready to run the training! Abort as you see fit. This may spam the output a bit.

In [ ]:
trainer = model.train
trainer(cfg)

In [ ]:
# Evaluate the final model
model.evaluate(cfg=cfg)

In [ ]:
model.save_model("latest_model.pth", cfg.save_path)